# Neste lab vamos utilizar uma Rede Multilayer Perceptron (MLP) para tentar classificar dígitos do dataset MNIST

In [ ]:
# Importando pacotes
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# Importando o dataset MNIST (http://yann.lecun.com/exdb/mnist/)
from tensorflow.examples.tutorials.mnist import input_data
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
# Veja abaixo alguns exemplos do dataset MNIST
from IPython.display import Image
Image(filename='08_mnist.jpg') 

Vamos definir alguns parâmetros para o treinamento da Rede Neural:
    
**learning_rate** - Taxa de aprendizado. Quanto menor, mais demorado será o treinamento, mas poderá encontrar os melhores valores
    
**training_epochs** - Quantas épocas de treinamento. O treinamento melhora com o avanço das épocas, entretanto se for muito grande poderá provocar o overfitting. Se for muito baixo, pode ocorrer o underfitting

**batch_size** - Quantidade da amostra utilizada em cada época de treinamento

In [ ]:
# Parâmetros do treinameno
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

O próximo passo será definir a estrutura da Rede Neural (MLP)

In [ ]:
# Camadas
n_hidden_1 = 256 # primeira camada oculta
n_hidden_2 = 256 # segunda camada oculta
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

Vamos definir dois tensores, um para os dados de entrada e outro para o de saída

In [ ]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

Veja que os dados de entrada consistem em um Array de duas dimensoes **[None, n_input]**

A primeira **None** diz respeito aos dados de entrada, que serão fornecidos depois. 

A segunda **n_input** foi fixada em 784, que corresponde à sequência de pixels de uma imagem MNIST de 28x28 pixels

Ou seja, neste método de classificação não está nos importanto o fator proximidade de pixels, mas apenas os pixels um depois do outro até terminar o arquivo da imagem.

Sabemos que esta não é a melhor forma de classificar imagens, mas é um início. Depois veremos e utilizaremos as CNNs

A segunda entrada é no formato **[None, n_classes]**

**None** diz respeito aos dados de saída esperada, que serão fornecidos depois. 

**n_classes** foi fixada em 10, visto que são apenas os caracteres numéricos de '0' a '9' que fazem parte desse dataset

Em seguida vamos criar uma função para facilitar a criação da Rede Neural


In [ ]:
# Função para criar a MLP
def multilayer_perceptron(x, weights, biases):
    # Camada oculta com ativação RELU 
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Camada oculta com ativação RELU 
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer com ativação linear
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

Agora vamos definir as variáveis que devem ser encontradas durante o treinamento da Rede Neural, após a passagem dos dados de entrada e saída previstos

Vamos definir aqui os dados de **peso** e **bias** para cada Perceptron (Neurônio) da rede MLP

In [ ]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

Perceba que acima definimos valores aleatórios para essas variáveis, pois precisam iniciar o treinamento com algum valor aleatório atribuído.

Com isso, a rede neural terá um desempenho ruim nas primeiras épocas, e vai melhorando com o tempo após aprender com os dados de treino

Em seguida vamos utlizar a função criada anteriormente para criar a MLP

In [ ]:
rede = multilayer_perceptron(x, weights, biases)

Como você já sabe, precisamos agora definir uma função de custo para dizer quão longe uma previsão está longe do valor correto.

In [ ]:
# Cost Function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = rede, labels = y))
tf.summary.scalar('cost', cost)

Além disso precisamos de uma função de otimização, que vai encontrar os valores de peso e bias que minimizam a função de custo

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

Pronto! Nosso blueprint da rede neural já está pronto:
- Definimos os valores de treinamento X e Y
- Definimos as variáveis a serem encontrada
- Definimos a estrutura da Rede Neural Multilayer Perceptron
- Definimos uma função de custo
- Definimos uma função de otimização

Agora podemos executar o modelo com o TensorFlow

In [ ]:
# Inicialziando as variáveis
init = tf.global_variables_initializer()


In [ ]:
# Criando variáveis para plotar o gráfico ao final
avg_cost_set = []
epoch_set = []
# Sessão
with tf.Session() as sess:
    # Inicializa as variaveis
    sess.run(init)
    merged = tf.summary.merge_all()
    now = datetime.now()
    writer = tf.summary.FileWriter('logs_08/'+now.strftime("%Y%m%d-%H%M%S"), sess.graph)
    # Ciclo de Treinamento
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop por todos os batches
        for i in range(total_batch):
            # Vamos utilizar uma função do próprio MNIST para buscar os N registros a serem utilizados em uma época
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Executa a otimização com o método run() do TF , passando os dados de treinamento X e Y conhecidos
            # Vejá que ignoramos o resultado do otimizador mas obtemos o resultado da função de custo 
            train_summary, _, c = sess.run([merged, optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
            # Computando a média do custo
            avg_cost += c / total_batch
        writer.add_summary(train_summary, epoch)
        avg_cost_set.append(avg_cost)
        epoch_set.append(epoch)
        # Display logs por epoch step
        if epoch % display_step == 0:
            print ("Epoca:", '%04d' % (epoch+1), "Custo = ", "{:.9f}".format(avg_cost))
    print ("Treinamento Concluído com ",training_epochs," épocas!")

    # Vamos plotar ao final o gráfico de custo por época de treinamento
    plt.plot(epoch_set,avg_cost_set, '-', label = 'Custo')
    plt.ylabel('Custo')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

    # Testando o modelo
    # Primeiro definimos uma para comparar o resultado da rede com os valores de y já conhecidos 
    correct_prediction = tf.equal(tf.argmax(rede, 1), tf.argmax(y, 1))
    # Em seguida calculando a acurácia
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Acurácia final: ", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Abra o **TensorBoard** utilizando o comando abaixo e veja os gráficos de custo e acurácia gerados:
    
python -m tensorboard.main --logdir="logs_08"

# Exercício:

Modifique os parâmetros de treinamento e rode novamente a rede para ver se consegue melhorar o resultado

Registre em uma planilha Excel os valores utilizados para cada treinamento e os valores de **custo** e **acurácia** final obtidos

## Responda:

- Qual a influência do learning_rate?
- Qual a influência do batch_size?
- Qual a influência da quantidade de épocas?